In [ ]:
import nltk
import numpy as np
import random
import string
import pandas as pd
# Instalaciones (solo 1 vez)
nltk.download('punkt')
nltk.download('wordnet') # Diccionario semantico incluido en NLTK
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Conectar con Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
PATH_DATA= '/content/drive/MyDrive/Colab Notebooks/nlp_introduction_course/data'
MODELS_DATA= '/content/drive/MyDrive/Colab Notebooks/nlp_introduction_course/models'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1 Leer Corpus

En este caso lo vamos a leer como un dataframe de pandas

In [ ]:
df = pd.read_csv(f'{PATH_DATA}/preguntas_respuesta_crucero.csv', sep=';')

In [ ]:
df

,pregunta,respuesta
0,cómo debo vestir,"Para saber cómo vestir, la estación del año y ..."
1,cuál es la ropa más adecuada,"Para saber cómo vestir, la estación del año y ..."
2,Que tipo de ropa debo llevar,"Para saber cómo vestir, la estación del año y ..."
3,qué vestimenta es conveniente llevar,"Para saber cómo vestir, la estación del año y ..."
4,"Para saber cómo vestir, la estación del año y ...","Para saber cómo vestir, la estación del año y ..."
...,...,...
148,viaje novios,Los cruceros han sido y siguen siendo una gran...
149,Los cruceros han sido y siguen siendo una gran...,Los cruceros han sido y siguen siendo una gran...
150,bebes,La edad mínima de los bebés para viajar en un ...
151,edad minima,La edad mínima de los bebés para viajar en un ...


Como podemos ver tiene 2 columnas, una para la pregunta y otra para la respuesta

Tambien podemos ver que varia preguntas, conducen a la misma respuesta, incluida la porpia restpuesta que tambien actua como pregunta

# 2.a Preprocesamiento del Texto con NTLK CORPUS

Pasamos todo a minusculas

In [ ]:
df['pregunta'] = df['pregunta'].str.lower() # Pasamos a minuscular

In [ ]:
df.head()

,pregunta,respuesta
0,cómo debo vestir,"Para saber cómo vestir, la estación del año y ..."
1,cuál es la ropa más adecuada,"Para saber cómo vestir, la estación del año y ..."
2,que tipo de ropa debo llevar,"Para saber cómo vestir, la estación del año y ..."
3,qué vestimenta es conveniente llevar,"Para saber cómo vestir, la estación del año y ..."
4,"para saber cómo vestir, la estación del año y ...","Para saber cómo vestir, la estación del año y ..."


Función para Tokenizar en frases

In [ ]:
def tokenizar_en_frases(dataframe,columna):
  return dataframe[columna].apply(nltk.sent_tokenize) # Tokenizado en frases

In [ ]:
tokenizar_en_frases(df,'pregunta')

0                                     [cómo debo vestir]
1                         [cuál es la ropa más adecuada]
2                         [que tipo de ropa debo llevar]
3                 [qué vestimenta es conveniente llevar]
4      [para saber cómo vestir, la estación del año y...
                             ...                        
148                                       [viaje novios]
149    [los cruceros han sido y siguen siendo una gra...
150                                              [bebes]
151                                        [edad minima]
152    [la edad mínima de los bebés para viajar en un...
Name: pregunta, Length: 153, dtype: object

Función para Tokenizar en palabras

In [ ]:
def tokenizar_en_palabras(dataframe,columna):
  return dataframe[columna].apply(nltk.word_tokenize) # Tokenizado en palabras

In [ ]:
tokenizar_en_palabras(df,'pregunta')

0                                   [cómo, debo, vestir]
1                    [cuál, es, la, ropa, más, adecuada]
2                    [que, tipo, de, ropa, debo, llevar]
3             [qué, vestimenta, es, conveniente, llevar]
4      [para, saber, cómo, vestir, ,, la, estación, d...
                             ...                        
148                                      [viaje, novios]
149    [los, cruceros, han, sido, y, siguen, siendo, ...
150                                              [bebes]
151                                       [edad, minima]
152    [la, edad, mínima, de, los, bebés, para, viaja...
Name: pregunta, Length: 153, dtype: object

Funciones para lematizar tokens y limpiar el texto

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemToken(tokens): # Función para lematizar la lista de tokens pasados
    return [lemmer.lemmatize(token) for token in tokens]


def LemNormalize(text): # Función para lematizar y normalizar el texto pasado como parámetro
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation) # Diccionario para los simbolos de puntuacion
    return LemToken(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
LemNormalize("Hola, como estas tronco?")

['hola', 'como', 'estas', 'tronco']

# 2.b Preprocesamiento de texto + 3 Evaluar la Similitud del Mensaje de Usuario vs Corpus

Importamos

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # para obtener el vecto de TF-IDF (Frecuencia de documentos de frecuencia inversa a termino)
from sklearn.metrics.pairwise import cosine_similarity # Similitud coseno para evaluar la distancia entre dos vectores
from nltk.corpus import stopwords

Preparamos los datos

In [ ]:
# Funcion para obtener una sola lista de una lista de listas
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
# Tokenizamos en frases 
sent_tokens = df['pregunta'].tolist()

Generamos una función para obtener la frase mas parecida, a una nueva peticion del usuario

In [ ]:
def generar_respuesta(dataframe,peticion,frases):
  bot_response = ''
  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('spanish')) # Creamos una instancia de TfidfVectorizer, pasandole el tokenizador y las stop words
  lfrases = list(sent_tokens)
  lfrases.append(peticion)
  tfidf = TfidfVec.fit_transform(lfrases) # Lo entrenamos a partir de una nueva lista, que contiene la peticion del usuario
  # Evaluamos la similitud coseno entre el mensaje del usuario tfidf[-1] y el corpus tfidf
  vals = cosine_similarity(tfidf[-1],tfidf) # Calculamos la similitud coseno entre la petición del usuario tfidf[-1] y el resto de corpus
  idx = vals.argsort()[0][-2] # Ordenamos, y obtenemos el Indice del valor mas cercano
  flat = vals.flatten() # Aplanamos la respuesta de la similitud coseno
  flat.sort() # la ordenamos
  req_tfidf = flat[-2] # y obtenemos el valor de la similitud
  if (req_tfidf==0): # Ningun vector esta proximo
        bot_response = bot_response+"Lo siento, no puedo entenderte"
        return bot_response
  else: # La respuesta mas proxima
      bot_response = bot_response+dataframe.iloc[idx]['respuesta']
      return bot_response

# 4. Definición de coincidencias Manuales (intenciones)

In [ ]:
SALUDOS_INPUTS = ("hola","buenas","saludos","que tal","hey","buenos días")
SALUDOS_OUTPUTS = ("Hola","Hola, ¿Que tal?","Hola ¿Como te puedo ayudar?","Hola, encantado de hablar contigo")

def saludos(sentence):
    for word in sentence.split():
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)
        
GRACIAS_INPUTS = ("gracias","muchas gracias","gracias por ayudarme")
GRACIAS_OUTPUTS = ("De nada","Me encanta ayudar","Es mi trabajo","Gracias a ti")

def agradecer(sentence):
    for word in sentence.split():
        if word.lower() in GRACIAS_INPUTS:
            return random.choice(GRACIAS_OUTPUTS)
        
        
CHISTE_INPUTS = ("chiste")
CHISTE_OUTPUTS = ("¿Por qué estás hablando con esas zapatillas?\nPorque pone 'converse'","¿Por qué las focas del circo miran siempre hacia arriba?\nPorque es donde están los focos.","Me da un café con leche corto.\nSe me ha roto la máquina, cambio.",
                   "¡Camarero! Este filete tiene muchos nervios.\nNormal, es la primera vez que se lo comen.","¡Camarero! ¿Qué le dice un techo a otro?.\nTecho de menos.")

def chiste(sentence):
    if "chiste" in sentence.lower().split():
        return random.choice(CHISTE_OUTPUTS)
    
def gestionar_intenciones(sentence):
    return saludos(sentence) or agradecer(sentence) or chiste(sentence)
        

In [ ]:
print(gestionar_intenciones('chiste'))

¿Por qué estás hablando con esas zapatillas?
Porque pone 'converse'


# 5 Generar respuesta del ChatBot

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
flag = True
print(bcolors.OKBLUE +"BOT: Mi nombre es ChatBot, Contestare a tus preguntas acerca de tus vaciones en el crucero. Si quieres salir escribe 'salir'"+ bcolors.ENDC)
while (flag==True):
    user_response = input() # Capturamos texto del usuario
    user_response = user_response.lower() # Convertimos a minuscula
    
    if user_response!="salir":
        
        intent_response = gestionar_intenciones(user_response)
        if intent_response is not None:
            print(bcolors.OKBLUE +"BOT: "+ intent_response+ bcolors.ENDC)
        else: # Si la palabra no es una intención, busco en el corpus
            print(bcolors.OKBLUE +"BOT: ",end="")
            print(generar_respuesta(df,user_response,sent_tokens) + bcolors.ENDC)
    else:
        flag = False
        print(bcolors.OKBLUE +"BOT: Nos vemos pronto..."+ bcolors.ENDC)
            

BOT: Mi nombre es ChatBot, Contestare a tus preguntas acerca de tus vaciones en el crucero. Si quieres salir escribe 'salir'
hola
BOT: Hola, encantado de hablar contigo
saber algun chiste
BOT: ¿Por qué las focas del circo miran siempre hacia arriba?
Porque es donde están los focos.
otro chiste
BOT: ¿Por qué estás hablando con esas zapatillas?
Porque pone 'converse'
pueden ir perros en el crucero?
BOT: Por norma general, las embarazadas que lleguen a la 23ª semana de gestación antes de que termine el crucero, podrán realizarlo, siempre y cuando tengan un certificado firmado por el especialista que declare que están en condiciones de realizar el viaje (el número de semanas variará en función de la naviera).
perro 
BOT: Lo siento, no puedo entenderte
gato
BOT: Lo siento, no puedo entenderte
mascotas
BOT: No se admiten mascotas a bordo.
perros
BOT: No se admiten mascotas a bordo.
gatos
BOT: No se admiten mascotas a bordo.
perro
BOT: Lo siento, no puedo entenderte


KeyboardInterrupt: ignored